In [0]:
df= spark.table("retail_bronze.superstore_bronze.gmart_bronze")
display(df.count())

In [0]:
from pyspark.sql.functions import *
df=df.withColumn("Quantity", (col("Quantity").try_cast("integer")))\
        .withColumn("Sales",col("Sales").cast("double"))\
            .withColumn("Discount",col("Discount").cast("double"))\
                .withColumn("Order Date",col("Order Date").cast("Timestamp"))\
                    .withColumn("Ship Date",col("Ship Date").cast("Timestamp"))\
                        .where(col("Quantity") >= 1)\
                            .na.drop(subset=["City","Postal Code"])
                        
            
display(df.count())

In [0]:
# Rename columns to remove spaces and special characters
df_clean = df.withColumnRenamed("Customer ID", "Customer_ID") \
    .withColumnRenamed("Customer Name", "Customer_Name") \
    .withColumnRenamed("Postal Code", "Postal_Code")


df_clean.groupBy(
        "Customer_ID", "Customer_Name", "City", "State", "Postal_Code", "Region", "Country", "Segment", "ingestion_time"
    ).agg(
        count("*").alias("Duplicatecount")
    ).orderBy(
        col("Duplicatecount").desc()
    ).select(
        "Customer_ID", "Customer_Name", "City", "State", "Postal_Code", "Region", "Country", "Segment", "ingestion_time"
    )


df_clean.groupBy(
    "Customer_ID", "Customer_Name", "City", "State", "Postal_Code", "Region", "Country", "Segment", "ingestion_time"
).agg(
    count("*").alias("Duplicatecount")
).orderBy(
    col("Duplicatecount").desc()
).select(
    "Customer_ID", "Customer_Name", "City", "State", "Postal_Code", "Region", "Country", "Segment", "ingestion_time"
).write.format('delta') \
    .mode("append") \
    .option("overwriteSchema", "true") \
    .saveAsTable("retail_silver.superstore_silver.customers_silver")
